In [2]:
import pandas as pd
from shapely.geometry import  Point
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing as prepro
from sklearn.model_selection import train_test_split
import seaborn as sns
from matplotlib import cm
import urllib.request
import shutil
import zipfile
import os
import re
from matplotlib import pyplot as plt



In [3]:
pdir_data = pd.read_csv("..\project 4\Police_Department_Incident_Reports__2018_to_Present (1).csv")


In [4]:
# look at a few rows of the pdir_data file.
pdir_data.sample(5)

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,...,Longitude,Point,Neighborhoods,ESNCAG - Boundary File,Central Market/Tenderloin Boundary Polygon - Updated,Civic Center Harm Reduction Project Boundary,HSOC Zones as of 2018-06-05,Invest In Neighborhoods (IIN) Areas,Current Supervisor Districts,Current Police Districts
648387,2020/08/06 08:40:00 PM,2020/08/06,20:40,2020,Thursday,2020/08/06 08:40:00 PM,95059175025,950591,200471558,202193114.0,...,-122.388673,POINT (-122.38867281318191 37.72704414579369),86.0,NaN,NaN,NaN,NaN,NaN,9.0,2.0
736765,2021/05/19 10:00:00 PM,2021/05/19,22:00,2021,Wednesday,2021/05/20 10:34:00 AM,103217106224,1032171,210309858,211401105.0,...,-122.404876,POINT (-122.40487561154785 37.794859532228344),104.0,NaN,NaN,NaN,NaN,NaN,3.0,6.0
68322,2021/07/11 01:37:00 PM,2021/07/11,13:37,2021,Sunday,2021/07/11 01:45:00 PM,104937364085,1049373,210436940,211921514.0,...,-122.406436,POINT (-122.40643633691012 37.77785653620884),32.0,NaN,NaN,1.0,1.0,NaN,10.0,1.0
23573,2023/05/16 10:00:00 PM,2023/05/16,22:00,2023,Tuesday,2023/05/16 10:32:00 PM,127649628150,1276496,230339841,231363349.0,...,-122.376640,POINT (-122.37664023199723 37.82908934030154),36.0,NaN,NaN,NaN,NaN,NaN,10.0,1.0
670545,2020/06/26 08:50:00 PM,2020/06/26,20:50,2020,Friday,2020/06/26 08:58:00 PM,93872628150,938726,200383947,201783028.0,...,-122.431477,POINT (-122.43147653287795 37.7474341037064),84.0,NaN,NaN,NaN,NaN,NaN,5.0,3.0


In [5]:
pdir_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 745686 entries, 0 to 745685
Data columns (total 35 columns):
 #   Column                                                Non-Null Count   Dtype  
---  ------                                                --------------   -----  
 0   Incident Datetime                                     745686 non-null  object 
 1   Incident Date                                         745686 non-null  object 
 2   Incident Time                                         745686 non-null  object 
 3   Incident Year                                         745686 non-null  int64  
 4   Incident Day of Week                                  745686 non-null  object 
 5   Report Datetime                                       745686 non-null  object 
 6   Row ID                                                745686 non-null  int64  
 7   Incident ID                                           745686 non-null  int64  
 8   Incident Number                             

In [6]:
# get the number of missing data points per column
missing_values_count = pdir_data.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count[0:10]

Incident Datetime            0
Incident Date                0
Incident Time                0
Incident Year                0
Incident Day of Week         0
Report Datetime              0
Row ID                       0
Incident ID                  0
Incident Number              0
CAD Number              168715
dtype: int64

In [7]:
# how many total missing values do we have?
total_cells = np.product(pdir_data.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
(total_missing/total_cells) * 100

17.619319660017755

In [8]:
# remove all the rows that contain a missing value
pdir_data.dropna()

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,...,Longitude,Point,Neighborhoods,ESNCAG - Boundary File,Central Market/Tenderloin Boundary Polygon - Updated,Civic Center Harm Reduction Project Boundary,HSOC Zones as of 2018-06-05,Invest In Neighborhoods (IIN) Areas,Current Supervisor Districts,Current Police Districts


In [9]:
# remove all columns with at least one missing value
columns_with_na_dropped = pdir_data.dropna(axis=1)
columns_with_na_dropped.head()

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,Report Type Code,Report Type Description,Incident Code,Incident Description,Resolution,Police District
0,2023/03/13 11:41:00 PM,2023/03/13,23:41,2023,Monday,2023/03/13 11:41:00 PM,125373607041,1253736,230167874,VS,Vehicle Supplement,7041,"Vehicle, Recovered, Auto",Open or Active,Out of SF
1,2023/03/01 05:02:00 AM,2023/03/01,05:02,2023,Wednesday,2023/03/11 03:40:00 PM,125379506374,1253795,236046151,II,Coplogic Initial,6374,"Theft, Other Property, >$950",Open or Active,Mission
2,2023/03/13 01:16:00 PM,2023/03/13,13:16,2023,Monday,2023/03/13 01:17:00 PM,125357107041,1253571,220343896,VS,Vehicle Supplement,7041,"Vehicle, Recovered, Auto",Open or Active,Out of SF
3,2023/03/13 10:59:00 AM,2023/03/13,10:59,2023,Monday,2023/03/13 11:00:00 AM,125355107041,1253551,230174885,VS,Vehicle Supplement,7041,"Vehicle, Recovered, Auto",Open or Active,Out of SF
4,2023/03/14 06:44:00 PM,2023/03/14,18:44,2023,Tuesday,2023/03/14 06:45:00 PM,125402407041,1254024,230176728,VS,Vehicle Supplement,7041,"Vehicle, Recovered, Auto",Open or Active,Out of SF


In [10]:
# just how much data did we lose?
print("Columns in original dataset: %d \n" % pdir_data.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

Columns in original dataset: 35 

Columns with na's dropped: 15


In [11]:
pdir_data.duplicated().sum()

0

In [12]:
pdir_data.describe().T

,count,mean,std,min,25%,50%,75%,max
Incident Year,745686.0,2.020199e+03,1.634525e+00,2.018000e+03,2.019000e+03,2.020000e+03,2.022000e+03,2.023000e+03
Row ID,745686.0,9.574947e+10,1.943477e+10,6.186871e+10,7.894054e+10,9.573509e+10,1.126127e+11,1.294467e+11
Incident ID,745686.0,9.574944e+05,1.943477e+05,6.186870e+05,7.894050e+05,9.573505e+05,1.126127e+06,1.294466e+06
Incident Number,745686.0,2.035874e+08,1.682296e+07,0.000000e+00,1.903007e+08,2.006171e+08,2.201405e+08,9.814243e+08
CAD Number,576971.0,2.044501e+08,2.487509e+07,1.000000e+00,1.910509e+08,2.024808e+08,2.204832e+08,1.000000e+09
Incident Code,745686.0,2.453040e+04,2.564006e+04,1.000000e+03,6.244000e+03,7.041000e+03,5.104000e+04,7.503000e+04
CNN,705464.0,2.531574e+07,3.099386e+06,2.001300e+07,2.395300e+07,2.492500e+07,2.647200e+07,5.420300e+07
Supervisor District,705070.0,5.930937e+00,2.839190e+00,1.000000e+00,3.000000e+00,6.000000e+00,9.000000e+00,1.100000e+01
Supervisor District 2012,705342.0,5.974387e+00,2.809070e+00,1.000000e+00,3.000000e+00,6.000000e+00,8.000000e+00,1.100000e+01
Latitude,705464.0,3.776912e+01,2.451145e-02,3.770799e+01,3.775445e+01,3.777579e+01,3.778579e+01,3.782999e+01


In [14]:

bc_data = pd.read_csv("..\project 4\BaltimoreCrimeData.csv")

C:\Users\lulum\AppData\Local\Temp\ipykernel_15592\948872916.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  bc_data = pd.read_csv("..\project 4\BaltimoreCrimeData.csv")


In [15]:
# look at a few rows of the pdir_data file.
bc_data.sample(5)

,OBJECTID,CCNO,CrimeDateTime,CrimeCode,Location,Description,Inside_Outside,Weapon,Post,Gender,Age,Race,Ethnicity,District,Neighborhood,Latitude,Longitude,GeoLocation,Premise,Total_Incidents
521388,521389,11K06207,2011/11/13 01:00:00+00,4E,400 OXFORD CT,COMMON ASSAULT,I,NaN,123.0,F,49.0,BLACK_OR_AFRICAN_AMERICAN,NaN,CENTRAL,UPTON,39.300091,-76.625533,"(39.300091,-76.625533)",ROW/TOWNHOUSE-OCC,1
362432,362433,15E00532,2015/05/01 18:00:00+00,5D,1500 N BENTALOU ST,BURGLARY,I,NaN,723.0,NaN,NaN,UNKNOWN,NaN,WESTERN,COPPIN HEIGHTS/ASH-CO-EAST,39.305177,-76.653917,"(39.305177012460604,-76.6539172227438)",RECREATION FACILITY,1
564167,564168,11A06033,2011/01/15 17:30:00+00,6D,400 N LINWOOD AVE,LARCENY FROM AUTO,O,NaN,221.0,F,52.0,WHITE,NaN,SOUTHEAST,MCELDERRY PARK,39.295912,-76.576504,"(39.295912,-76.576504)",STREET,1
42387,42388,22E07593,2022/05/23 20:00:00+00,6J,5600 BELAIR RD,LARCENY,Outside,NaN,426.0,NaN,NaN,NaN,NaN,NORTHEAST,WALTHERSON,39.341927,-76.546298,"(39.341926999999998,-76.546297999999993)",CAR LOT-NEW/USED,1
557720,557721,11C12079,2011/03/23 12:30:00+00,6E,6700 PULASKI HWY,LARCENY,O,NaN,441.0,M,25.0,BLACK_OR_AFRICAN_AMERICAN,NaN,NORTHEAST,PULASKI INDUSTRIAL AREA,39.305631,-76.537192,"(39.305631,-76.537192)",YARD/BUSINESS,1


In [16]:
bc_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568614 entries, 0 to 568613
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   OBJECTID         568614 non-null  int64  
 1   CCNO             568614 non-null  object 
 2   CrimeDateTime    568614 non-null  object 
 3   CrimeCode        568614 non-null  object 
 4   Location         565801 non-null  object 
 5   Description      568614 non-null  object 
 6   Inside_Outside   466659 non-null  object 
 7   Weapon           129754 non-null  object 
 8   Post             560474 non-null  float64
 9   Gender           476867 non-null  object 
 10  Age              454448 non-null  float64
 11  Race             554983 non-null  object 
 12  Ethnicity        28422 non-null   object 
 13  District         560530 non-null  object 
 14  Neighborhood     560359 non-null  object 
 15  Latitude         567710 non-null  float64
 16  Longitude        567710 non-null  floa

In [17]:
# get the number of missing data points per column
missing_values_count = bc_data.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count[0:10]

OBJECTID               0
CCNO                   0
CrimeDateTime          0
CrimeCode              0
Location            2813
Description            0
Inside_Outside    101955
Weapon            438860
Post                8140
Gender             91747
dtype: int64

In [18]:
# how many total missing values do we have?
total_cells = np.product(pdir_data.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
(total_missing/total_cells) * 100

5.486575927592656

In [19]:
# remove all the rows that contain a missing value
bc_data.dropna()

,OBJECTID,CCNO,CrimeDateTime,CrimeCode,Location,Description,Inside_Outside,Weapon,Post,Gender,Age,Race,Ethnicity,District,Neighborhood,Latitude,Longitude,GeoLocation,Premise,Total_Incidents
25115,25116,22J05492,2022/10/17 20:20:00+00,3AJF,800 PEACH ST,ROBBERY - CARJACKING,Inside,HANDGUN,941.0,M,25.0,WHITE,NOT_HISPANIC_OR_LATINO,SOUTHERN,OTTERBEIN,39.280150,-76.617167,"(39.280149531985785,-76.61716701606052)",STREET,1
25136,25137,22I07653,2022/09/24 05:51:00+00,4A,1300 N ELLWOOD AVE,AGG. ASSAULT,Inside,AUTOMATIC_HANDGUN,332.0,M,59.0,BLACK_OR_AFRICAN_AMERICAN,UNKNOWN,EASTERN,BEREA,39.306226,-76.573719,"(39.306226000000002,-76.573718999999997)",STREET,1
25139,25140,22I07887,2022/09/24 21:20:00+00,3AO,3000 E PRATT ST,ROBBERY,Inside,PERSONAL_WEAPONS,222.0,M,22.0,ASIAN,UNKNOWN,SOUTHEAST,PATTERSON PARK NEIGHBORHOOD,39.290054,-76.574555,"(39.290053999999998,-76.574555000000004)",STREET,1
25146,25147,22J01594,2022/10/06 00:30:00+00,3AK,300 S CAROLINE ST,ROBBERY,Inside,KNIFE_CUTTING_INSTRUMENT,211.0,F,23.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHEAST,PERKINS HOMES,39.287397,-76.596972,"(39.287397401300908,-76.596972027745238)",STREET,1
25155,25156,22J01835,2022/10/06 18:23:00+00,3NF,2900 PHELPS LN,ROBBERY,Inside,HANDGUN,835.0,M,13.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHWEST,CARROLL-SOUTH HILTON,39.288772,-76.669304,"(39.288772000000002,-76.669303999999997)",PARK,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554266,554267,22D04832,2011/04/17 15:00:00+00,4E,1600 RAMBLEWOOD RD,COMMON ASSAULT,Outside,PERSONAL_WEAPONS,415.0,M,61.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,NORTHEAST,LOCH RAVEN,39.360196,-76.585243,"(39.360196000000002,-76.585243000000006)",OTHER/RESIDENTIAL,1
564875,564876,22G08951,2002/07/29 10:50:00+00,4E,2700 BAKER ST,COMMON ASSAULT,Inside,PERSONAL_WEAPONS,723.0,M,62.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,WESTERN,COPPIN HEIGHTS/ASH-CO-EAST,39.306188,-76.660960,"(39.306187999999999,-76.660960000000003)",STREET,1
564878,564879,22F00864,2002/06/03 14:45:00+00,6F,1700 E LOMBARD ST,LARCENY,Outside,PERSONAL_WEAPONS,212.0,M,23.0,WHITE,UNKNOWN,SOUTHEAST,UPPER FELLS POINT,39.290369,-76.592460,"(39.290368999999998,-76.592460000000003)",RESTAURANT,1
564881,564882,22D08283,2002/04/28 22:00:00+00,4E,500 S Luzerne Ave,COMMON ASSAULT,Outside,PERSONAL_WEAPONS,214.0,F,35.0,WHITE,UNKNOWN,SOUTHEAST,CANTON,39.285544,-76.579935,"(39.285544351713874,-76.579935276084086)",OTHER/RESIDENTIAL,1


In [20]:
# remove all columns with at least one missing value
columns_with_na_dropped =bc_data.dropna(axis=1)
columns_with_na_dropped.head()

,OBJECTID,CCNO,CrimeDateTime,CrimeCode,Description,GeoLocation,Total_Incidents
0,1,23F08231,2023/06/24 04:01:00+00,4B,AGG. ASSAULT,"(39.27330200992213,-76.69243902745305)",1
1,2,23F08231,2023/06/24 04:01:00+00,4B,AGG. ASSAULT,"(39.27330200992213,-76.69243902745305)",1
2,3,23F08231,2023/06/24 04:01:00+00,4B,AGG. ASSAULT,"(39.27330200992213,-76.69243902745305)",1
3,4,23F08231,2023/06/24 04:01:00+00,3JK,ROBBERY,"(39.27330200992213,-76.69243902745305)",1
4,5,23F08235,2023/06/24 03:45:00+00,5A,BURGLARY,"(39.24643210111462,-76.63681903810716)",1


In [21]:
# just how much data did we lose?
print("Columns in original dataset: %d \n" % bc_data.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

Columns in original dataset: 20 

Columns with na's dropped: 7


In [22]:
bc_data.duplicated().sum()

0

In [25]:
bc_data.to_csv("..\project 4\BaltimoreCrimeData.csv")